In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_5"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
7223  317.475875               0          0          1          0         0   
7224  383.628682               0          0          1          0         0   
7225  305.523072               0          0          0          1         0   
7226  308.408168               0          0          1          0         0   
7227  355.518770               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
9037  297.867853               0          0          0          0         1   
9038  290.024007               0          0          0          0         1   
9039  335.823557               0          0          0          1         0   
9040  324.965348               0          0          0          0         1   
9041  261.341313               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Silvester                 0.752698
zwischen_den_jahren       0.365729
Sommer                    0.208068
Monat_8                   0.201454
is_holiday                0.200365
Samstag                   0.145132
Sonntag                   0.141359
Temp_Warm                 0.102382
Number_of_ships           0.101397
Number_of_ships_scaled    0.101397
Monat_7                   0.085772
Ship                      0.085769
Temp_Mild                 0.084154
Ostertag                  0.083630
Cloud_ok                  0.071720
Feiertag                  0.065401
KielerWoche               0.062338
Cloud_Partly_Cloudy       0.056453
Heimspiel                 0.045645
wetter_sehr_schön         0.045481
Monat_9                   0.041326
Monat_6                   0.038020
Wind_Moderate             0.031546
Cloud_Clear               0.027513
Weather_Light_Issues      0.016144
Temp_Hot                  0.010511
Wind_ok                   0.007144
Weather_Moderate    

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


VPI                       0.149337
Temp_Cold                 0.074531
Sommer                    0.065629
Werktag                   0.059722
Monat_8                   0.050012
Winter                    0.049842
Weihnachtsmarkt           0.045959
Monat_12                  0.044313
Sonntag                   0.039256
Temp_Mild                 0.032773
Wind_Moderate             0.031853
Montag                    0.026526
Monat_7                   0.026120
Monat_11                  0.025791
Weather_Light_Issues      0.022486
Silvester                 0.019237
is_holiday                0.017718
Monat_2                   0.016335
Donnerstag                0.015892
Wind_Light                0.015492
zwischen_den_jahren       0.015086
Samstag                   0.014473
Weather_Moderate          0.013799
Cloud_Clear               0.012591
Herbst                    0.012433
Cloud_ok                  0.012294
Cloud_Cloudy              0.012141
Cloud_Partly_Cloudy       0.011566
Temp_Warm           

In [4]:
# Features mit geringer linearer und nichtlinearer Korrelation finden
low_linear_corr_features = correlation_with_target[correlation_with_target.abs() < 0.1].index
low_nonlinear_corr_features = mi_series[mi_series < 0.02].index

# Schnittmenge der Features mit beiden Bedingungen
low_corr_features = set(low_linear_corr_features).intersection(low_nonlinear_corr_features)

# Ausgabe der Features
#print("Features mit geringer linearer und nichtlinearer Korrelation:", list(low_corr_features))

# DataFrame erstellen für die Ausgabe in Tabellenform
low_corr_features_table = pd.DataFrame({
    "Feature": list(low_corr_features),
    "Lineare Korrelation": [correlation_with_target[feature] for feature in low_corr_features],
    "Nichtlineare Korrelation": [mi_series[feature] for feature in low_corr_features]
})


# Tabelle nach der linearen Korrelation sortieren
low_corr_features_table = low_corr_features_table.sort_values(by="Lineare Korrelation", ascending = False)

# Ausgabe der sortierten Tabelle
print(low_corr_features_table)



                 Feature  Lineare Korrelation  Nichtlineare Korrelation
18                  Ship             0.085769                  0.000497
8               Ostertag             0.083630                  0.006341
17              Cloud_ok             0.071720                  0.012294
13              Feiertag             0.065401                  0.007289
2            KielerWoche             0.062338                  0.000000
21   Cloud_Partly_Cloudy             0.056453                  0.011566
14             Heimspiel             0.045645                  0.000000
24     wetter_sehr_schön             0.045481                  0.007106
6                Monat_9             0.041326                  0.010035
7                Monat_6             0.038020                  0.003120
33           Cloud_Clear             0.027513                  0.012591
25              Temp_Hot             0.010511                  0.007622
5                Wind_ok             0.007144                  0

In [5]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Umsatz', 'Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'Weihnachtsmarkt', 'Silvester', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  1.169477e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot  1.500577e+07
5              Cloud_Clear  8.145413e+05
6      Cloud_Partly_Cloudy  1.169889e+06
7             Cloud_Cloudy  1.091062e+05
8               Wind_Light  3.870857e+04
9            Wind_Moderate  1.168463e+05
10             Wind_Strong  2.113227e+06
11            Weather_Good  1.182944e+07
12    Weather_Light_Issues  8.581310e+04
13        Weather_Moderate  3.167107e+05
14          Weather_Severe  1.810797e+07
15             KielerWoche  1.453789e+00
16                  Montag  1.906687e+06
17                Dienstag  2.154440e+06
18                Mittwoch  5.825325e+05
19              Donnerstag  1.072312e+05
20                 Freitag  6.563315e+05
21                 Samstag  1.192699e+05
22                 Sonntag  2.503423e+06
23              

In [7]:
# suche miteinander korrelierrende features


In [8]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Wind_Strong', 'Wind_Moderate'), ('Weather_Light_Issues', 'Weather_Good'), ('Monat_4', 'Frühling'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Monat_10', 'Herbst'), ('Monat_9', 'Herbst'), ('Monat_5', 'Frühling'), ('Werktag', 'Sonntag'), ('Ship', 'Number_of_ships'), ('Monat_3', 'Frühling'), ('Sommer', 'Temp_Cold'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Monat_6', 'Sommer'), ('wetter_sehr_schön', 'Temp_Hot'), ('Wind_ok', 'Wind_Moderate'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Heimspiel', 'Samstag'), ('Wind_Moderate', 'Wind_Light'), ('Cloud_ok', 'Cloud_Clear'), ('Monat_12', 'Winter'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Monat_6', 'KielerWoche'), ('Monat_7', 'Sommer'), ('Monat_8', 'Sommer'), ('Wind_ok', 'Wind_Strong'), ('Cloud_ok', 'Cloud_Cloudy'), ('Monat_11', 'Herbst'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Winter', 'Temp_Cold'), ('Winter', 'Temp_Mild'), ('Sommer', 'Temp_Warm'), ('Monat_4', 'Markt'), ('Silvester', 'Umsatz'), ('Temp_M